<a href="https://colab.research.google.com/github/arminmu13106-art/TEAM7-Capstone-Project/blob/main/Final_approach_1__2_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Gen AI:

Since I can't import files into Colab, I asked AI if there are other methods: I can't import files into Colab. Please tell me if there are other methods.

See Data Tell me what positions and grades are listed in the document.

Improve optimization rate Please optimize the keywords for me to improve the prediction accuracy.

Model Search Provide me with models that may be suitable.

In [ ]:
  # Rule-based matching (baseline): Identify relevant job titles and text passages using predefined label lists and assign domain and seniority accordingly.
import pandas as pd
import json
from sklearn.metrics import accuracy_score
from google.colab import drive

# 1. Load Data
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Colab Notebooks/linkedin-cvs-annotated.json'

with open(file_path, 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

# 2. Flattening
# Create an empty list
flattened_records = []
#Open each resume one by one
for person_cv in raw_data:
    for job in person_cv:
        if job.get('status') == 'ACTIVE':
            flattened_records.append(job)
# Change the format
df = pd.DataFrame(flattened_records)

# 3. Define Rules
dept_rules = {
    'Information Technology': ['it', 'information technology', 'software', 'developer', 'engineer', 'systems', 'system administrator', 'network', 'architect','cloud', 'devops', 'data', 'infrastructure'],

    'Customer Support': ['customer support', 'support specialist', 'technical support', 'service desk', 'helpdesk', 'customer service', 'support engineer', 'maintenance', 'after sales'],

    'Consulting': ['consultant', 'consulting', 'advisory','management consulting', 'strategy consulting', 'technology consulting', 'medical consulting'],

    'Project Management': ['project manager', 'project management', 'program manager', 'quality management', 'delivery manager','project lead', 'development lead'],

    'Sales': ['sales', 'account manager', 'key account', 'sales manager', 'business sales', 'commercial', 'sales director'],

    'Business Development': ['business development', 'bd manager', 'growth', 'strategic partnership', 'partnerships', 'alliances'],

    'Administrative': ['administrative', 'administration', 'operations coordinator', 'office manager', 'business planning', 'system coordination'],

    'Marketing': ['marketing', 'digital marketing', 'online marketing', 'brand', 'content', 'seo', 'sem','graphic designer', 'art director', 'communications'],

    'Human Resources': ['human resources', 'hr', 'people', 'talent', 'recruitment', 'recruiter', 'hr manager', 'people operations'],

    'Purchasing': ['purchasing', 'procurement', 'buyer', 'sourcing', 'supply', 'demand manager', 'global demand', 'project accounting']
}

seniority_rules = {
    'Director': ['director', 'vice president', 'vp', 'board', 'partner'],
    'Management': ['manager', 'head', 'chief', 'executive', 'managing', 'supervisor', 'leiter'],
    'Lead': ['lead', 'team lead', 'tech lead', 'project lead'],
    'Senior': ['senior', 'sr', 'expert', 'principal', 'staff'],
    'Junior': ['junior', 'assistant', 'entry', 'intern', 'trainee', 'working student', 'student','apprentice']
}

# 4. Prediction Function
def rule_based_match(text, rules_dict, default_val):
    if pd.isna(text):
        return default_val
    # Standardized lowercasing
    text = str(text).lower()
    for label, keywords in rules_dict.items():
        for kw in keywords:
            if kw in text:
                return label
    return default_val

# 5. Apply Predictions
df['pred_dept'] = df['position'].apply(rule_based_match, args=(dept_rules, 'Other'))
df['pred_sen'] = df['position'].apply(rule_based_match, args=(seniority_rules, 'Professional'))

# 6. Results & Evaluation
print(f"Domain Prediction Accuracy: {accuracy_score(df['department'], df['pred_dept']):.2%}")
print(f"Seniority Prediction Accuracy: {accuracy_score(df['seniority'], df['pred_sen']):.2%}")

# 7. Result Showcase
print("\nPrediction Examples:")
print(df[['position', 'department', 'pred_dept', 'seniority', 'pred_sen']].head())

In [ ]:
#Embedding-based labeling: Use the provided label lists to generate embeddings (e.g., via LLMs or sentence transformers). Compute similarity between profile text and label embeddings and perform zero-shot classification.
import pandas as pd
import json
import torch
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics import accuracy_score

from google.colab import drive
drive.mount('/content/drive')
df_dept = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/department-v2.csv')
df_sen  = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/seniority-v2.csv')
file_path = '/content/drive/MyDrive/Colab Notebooks/linkedin-cvs-annotated.json'
with open(file_path, 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

records = []
for person_cv in raw_data:
    for job in person_cv:
        if job.get('status') == 'ACTIVE':
            records.append(job)

df_test = pd.DataFrame(records)

# Label Lists
dept_labels = sorted(df_dept['label'].unique().tolist())
sen_labels  = sorted(df_sen['label'].unique().tolist())

test_positions = df_test['position'].astype(str).tolist()

# Zero-shot
def predict_labels(text_embeds, label_embeds, label_list):
    sim_matrix = util.cos_sim(text_embeds, label_embeds)
    best_indices = torch.argmax(sim_matrix, dim=1).cpu().tolist()
    return [label_list[i] for i in best_indices]

# Modell List
basic_transformer_models = [
                     "all-mpnet-base-v2",
                     "all-MiniLM-L6-v2",
                     "multi-qa-mpnet-base-dot-v1",
                     "paraphrase-multilingual-mpnet-base-v2",
                     "distiluse-base-multilingual-cased-v2",
]

results = []

for model_name in basic_transformer_models:
    print(f"\Testing model: {model_name}")

    try:
        model = SentenceTransformer(model_name)

        # label embeddings
        dept_label_embeddings = model.encode(
            dept_labels, convert_to_tensor=True, normalize_embeddings=True
        )
        sen_label_embeddings = model.encode(
            sen_labels, convert_to_tensor=True, normalize_embeddings=True
        )

        # position embeddings
        position_embeddings = model.encode(
            test_positions,
            convert_to_tensor=True,
            show_progress_bar=True,
            normalize_embeddings=True
        )

        # predictions
        df_test['pred_department'] = predict_labels(
            position_embeddings, dept_label_embeddings, dept_labels
        )
        df_test['pred_seniority'] = predict_labels(
            position_embeddings, sen_label_embeddings, sen_labels
        )

        # evaluation
        dept_acc = accuracy_score(df_test['department'], df_test['pred_department'])
        sen_acc  = accuracy_score(df_test['seniority'], df_test['pred_seniority'])

        results.append({
            "model": model_name,
            "department_acc": dept_acc,
            "seniority_acc": sen_acc,
        })

        print(f"Department Acc: {dept_acc:.2%}")
        print(f"Seniority Acc: {sen_acc:.2%}")

    except Exception as e:
        print(f"Model {model_name} failed: {e}")


results_df = pd.DataFrame(results)

best_dept_idx = results_df['department_acc'].idxmax()
best_sen_idx  = results_df['seniority_acc'].idxmax()

best_dept_model = results_df.loc[best_dept_idx]
best_sen_model = results_df.loc[best_sen_idx]


print(f"Best Model for Department Classification:")
print(f"Model Name: {best_dept_model['model']}")
print(f"Accuracy: {best_dept_model['department_acc']:.2%}")

print(f"Best Model for Seniority Classification:")
print(f"Model Name: {best_sen_model['model']}")
print(f"Accuracy: {best_sen_model['seniority_acc']:.2%}")


It can be observed that the prediction success rate of Embedding-based labeling is lower than that of Rule-based matching (baseline). Even after attempting various models, its highest prediction accuracy remains below 35%. However, when the training set is split into 80% for training and 20% for testing, the success rate reaches about 80%. This suggests an occurrence of overfitting. Additionally, the emergence of new classifications in the test set is also a likely factor contributing to the prediction rate falling below 35%.

Extensions: The seniority level typically increases over time

In [ ]:
import pandas as pd
import json
import torch
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics import accuracy_score

from google.colab import drive
drive.mount('/content/drive')
df_dept = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/department-v2.csv')
df_sen  = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/seniority-v2.csv')
file_path = '/content/drive/MyDrive/Colab Notebooks/linkedin-cvs-annotated.json'
with open(file_path, 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

# Label Lists
dept_labels = sorted(df_dept['label'].unique().tolist())

# seniority Order
sen_labels = ["Junior", "Senior", "Lead", "Management", "Director"]

sen_order = {l: i for i, l in enumerate(sen_labels)}
inv_sen_order = {i: l for l, i in sen_order.items()}


# Zero-shot
def predict_labels(text_embeds, label_embeds, label_list):
    sim_matrix = util.cos_sim(text_embeds, label_embeds)
    best_indices = torch.argmax(sim_matrix, dim=1).cpu().tolist()
    return [label_list[i] for i in best_indices]

# Modell List
basic_transformer_models = [
                     "all-mpnet-base-v2",
                     "all-MiniLM-L6-v2",
                     "multi-qa-mpnet-base-dot-v1",
                     "paraphrase-multilingual-mpnet-base-v2",
                     "distiluse-base-multilingual-cased-v2",
]

results = []

for model_name in basic_transformer_models:
    print(f"\nTesting model: {model_name}")

    try:
        model = SentenceTransformer(model_name)

        # label embeddings
        dept_label_embeddings = model.encode(
            dept_labels, convert_to_tensor=True, normalize_embeddings=True
        )
        sen_label_embeddings = model.encode(
            sen_labels, convert_to_tensor=True, normalize_embeddings=True
        )

        y_true_dept = []
        y_pred_dept = []
        y_true_sen = []
        y_pred_sen = []

        for person_cv in raw_data:
            # time based order
            jobs = sorted(person_cv,key=lambda x: x.get("start_date", ""))

            texts = [str(j.get("position", "")) for j in jobs]

            embeds = model.encode(
                texts,
                convert_to_tensor=True,
                normalize_embeddings=True
            )

            curr_dept_preds = predict_labels(embeds, dept_label_embeddings, dept_labels)
            curr_sen_preds = predict_labels(embeds, sen_label_embeddings, sen_labels)

            numeric_sen_preds = [sen_order[p] for p in curr_sen_preds]

            active_idx = None
            for i, j in enumerate(jobs):
                if j.get("status") == "ACTIVE":
                    active_idx = i
                    break

            if active_idx is None:
                continue

            if active_idx:
                constrained_val = max(numeric_sen_preds[:active_idx + 1])
                final_sen_pred = inv_sen_order[constrained_val]

            else:
                final_sen_pred = curr_sen_preds[active_idx]

            # 5. data collection
            true_dept = jobs[active_idx].get("department")
            true_sen  = jobs[active_idx].get("seniority")

            # skip if missing annotations
            if true_dept is None or true_sen is None:
              continue

            y_true_dept.append(str(true_dept).strip())
            y_pred_dept.append(str(curr_dept_preds[active_idx]).strip())

            y_true_sen.append(str(true_sen).lower().strip())
            y_pred_sen.append(str(final_sen_pred).lower().strip())

        # Accuracy
        dept_acc = accuracy_score(y_true_dept, y_pred_dept)
        sen_acc_2 = accuracy_score(y_true_sen, y_pred_sen)

        results.append({
            "model": model_name,
            "department_acc": dept_acc,
            "seniority_acc_2": sen_acc_2,
        })

        print(f"Department Acc: {dept_acc:.2%}")
        print(f"Seniority Acc 2: {sen_acc_2:.2%}")

    except Exception as e:
        print(f"Model {model_name} failed: {e}")

# evaluation
results_df = pd.DataFrame(results)

best_dept_idx = results_df['department_acc'].idxmax()
best_sen_idx  = results_df['seniority_acc_2'].idxmax()

best_dept_model = results_df.loc[best_dept_idx]
best_sen_model = results_df.loc[best_sen_idx]

print(f"Best Model for Department Classification:")
print(f"Model Name: {best_dept_model['model']}")
print(f"Accuracy: {best_dept_model['department_acc']:.2%}")

print(f"Best Model for Seniority Classification (Method 2):")
print(f"Model Name: {best_sen_model['model']}")
print(f"Accuracy: {best_sen_model['seniority_acc_2']:.2%}")

The above methods assume that job seniority levels generally do not decrease. We observe that accuracy improvements do not exceed 1%, and the optimal models remain unchanged. Therefore, paraphrase-multilingual-mpnet-base-v2 and multi-qa-mpnet-base-dot-v1 are identified as the optimal models for predicting Department and Seniority, respectively. When utilizing these models, the assumption that job seniority levels typically do not decrease can be disregarded.